https://www.kaggle.com/berkayalan/vaccinations-in-united-states
us_vaccinations.csv

# The main goal of this dashboard is to display the pogress of vaccinations in the 52 United states

## in this notebook I will eplore the best way to display the data. Later I will make a django app where I am aiming to display the following views:
* A heatmap of the united states to keep track of the amount of vacinnations in each state
* A linechart for each state filtered by state and date to diaplay the change in vaccinations
* A bar chart of pie chart to display the amount of people vaccinted compared to the total population per state

*Seaborn: Size of the state == box --> vaccination vs total population filling up the box

# Imports

In [10]:
import pandas as pd
import sqlite3 as sql
from bokeh.io import push_notebook, output_notebook 
from bokeh.plotting import figure,show
from bokeh.embed import components
from bokeh.models import ColumnDataSource, HoverTool,Legend
# from bokeh.palettes import Magma,RdYlGn
from bokeh.transform import factor_cmap

In [11]:
output_notebook() #to see notebook in the app

Loading BokehJS ...

## Opening dataset that contains vaccine information per state

In [12]:
db = '../dash/project_2.db'
conn = sql.connect(db)

In [13]:
vacc = pd.read_sql("SELECT * from vacc", conn, index_col='index')
vacc.head()

,date,location,total_vaccinations,total_distributed,people_vaccinated,people_fully_vaccinated_per_hundred,total_vaccinations_per_hundred,people_fully_vaccinated,people_vaccinated_per_hundred,distributed_per_hundred,daily_vaccinations_raw,daily_vaccinations,daily_vaccinations_per_million,share_doses_used
index,,,,,,,,,,,,,,
0,12.01.2021,Alabama,78134.0,377025.0,70861.0,0.15,21551.0,7270.0,16438.0,25385.0,0.0,0.0,0.0,0.207
1,13.01.2021,Alabama,84040.0,378975.0,74792.0,0.19,25934.0,9245.0,19360.0,26846.0,5906.0,5906.0,1205.0,0.222
2,14.01.2021,Alabama,92300.0,435350.0,80480.0,0.19,32143.0,9245.0,23377.0,32356.0,8260.0,7083.0,1445.0,0.212
3,15.01.2021,Alabama,100567.0,444650.0,86956.0,0.28,44318.0,13488.0,28126.0,44386.0,8267.0,7478.0,1525.0,0.226
4,16.01.2021,Alabama,100567.0,444650.0,86956.0,0.28,44318.0,13488.0,28126.0,44386.0,7557.0,7498.0,1529.0,0.226


In [32]:
vacc[vacc['location'] == 'Vermont']

,date,location,total_vaccinations,total_distributed,people_vaccinated,people_fully_vaccinated_per_hundred,total_vaccinations_per_hundred,people_fully_vaccinated,people_vaccinated_per_hundred,distributed_per_hundred,daily_vaccinations_raw,daily_vaccinations,daily_vaccinations_per_million,share_doses_used
index,,,,,,,,,,,,,,
8744,12.01.2021,Vermont,25757.0,64925.0,1486262.0,34.18,41365.00,1095829.0,46.36,44296.00,0.00,0.0,0.0,0.397
8745,13.01.2021,Vermont,29571.0,69800.0,1486262.0,34.18,27120.00,1095829.0,46.36,43770.00,3814.00,3814.0,6112.0,0.424
8746,14.01.2021,Vermont,29571.0,69800.0,26603.0,34.18,27120.00,1095829.0,46113.00,43770.00,0.00,1907.0,3056.0,0.424
8747,15.01.2021,Vermont,32470.0,74000.0,28355.0,0.66,44232.00,4100.0,19815.00,31717.00,2899.00,2238.0,3587.0,0.439
8748,16.01.2021,Vermont,32470.0,74000.0,28355.0,0.66,44232.00,4100.0,19815.00,31717.00,2027.75,2185.0,3502.0,0.439
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8892,9.06.2021,Vermont,801943.0,905680.0,447542.0,59.46,128.52,371051.0,71.72,145.14,716.00,3307.0,5300.0,0.885
8893,10.06.2021,Vermont,804718.0,906380.0,448270.0,59.81,128.96,373214.0,71.84,145.26,2775.00,3473.0,5566.0,0.888
8894,11.06.2021,Vermont,804718.0,908200.0,448298.0,59.82,128.96,373274.0,71.84,145.55,0.00,3047.0,4883.0,0.886


In [28]:
total = pd.read_sql("SELECT * from total", conn, index_col='index')
total.sort_values(by='perc_daily_vacc_pop', ascending=False)

,state,latitude,longitude,location,state_population,daily_vaccinations,doses_needed,perc_daily_vacc_pop
index,,,,,,,,
47,VT,44.558803,-72.577841,Vermont,643077,787136,1286154,61
19,MA,42.407211,-71.382437,Massachusetts,7029917,8229843,14059834,58
7,DC,38.905985,-77.033418,District of Columbia,689545,788817,1379090,57
6,CT,41.603221,-73.087749,Connecticut,3605944,4048765,7211888,56
21,ME,45.253783,-69.445469,Maine,1362359,1484927,2724718,54
11,HI,19.898682,-155.665857,Hawaii,1455271,1600159,2910542,54
40,RI,41.580095,-71.477429,Rhode Island,1097379,1169226,2194758,53
30,NH,43.193852,-71.572395,New Hampshire,1377529,1417612,2755058,51
20,MD,39.045755,-76.641271,Maryland,6177224,6289805,12354448,50


In [ ]:
each state: Range(smallest 1/2million, largest 40million)
    x: numpy array of 50 total # of doses
    y: numpy array of 50 elements containing the percent vaccinated

In [15]:
state_doses   = dict(zip(total.location,total.doses_needed))

In [16]:
state = 'Alabama'

In [17]:
# new_vacc = vacc[vacc['location']== state][['date','total_vaccinations']][-7:]
# new_vacc['total_vaccinations in Millions']= new_vacc['total_vaccinations']//1000

In [18]:
# state_doses[state]

In [19]:
# new_vacc['percentage'] = new_vacc['total_vaccinations'] / state_doses[state] * 100

In [20]:
# new_vacc['percentage'] = new_vacc['percentage'].astype(int)

In [21]:
# source = ColumnDataSource(new_vacc)

In [22]:
'''We have to make a note that 2 shots need to be given for a person to be considered fully vaccinated. 

Although Alabama has more shots in the state. The Amount of people is very large so only 31% of the 
total population was vaccinated'''
def line_chart_1(state):
    new_vacc = vacc[vacc['location']== state][['date','total_vaccinations']][-7:]
    new_vacc['total_vaccinations in Millions']= new_vacc['total_vaccinations']//1000
    new_vacc['percentage'] = new_vacc['total_vaccinations'] / state_doses[state] * 100
    source = ColumnDataSource(new_vacc)
    
    dates = source.data['date'].tolist()
    p = figure(x_range=dates, title= f'Total Vaccinations in {state} (last 7 days recorded)')
    p.line(x='date', y='total_vaccinations in Millions', source=source, legend_label="Number of Vaccinations", line_width=8,line_color="green")
    p.add_tools(HoverTool())
    p.yaxis.axis_label = 'Total Number of Vaccinations (Thousands)'
    p.xaxis.axis_label = 'Date (Day.Month.Year)'
    p.legend.location = "top_left"

    show(p)
    return p

line_chart_1(state)

Figure(id='1003', ...)

In [23]:
def line_chart_2(state):
    new_vacc = vacc[vacc['location']== state][['date','total_vaccinations']][-7:]
    new_vacc['total_vaccinations in Millions']= new_vacc['total_vaccinations']//1000
    new_vacc['percentage'] = new_vacc['total_vaccinations'] / state_doses[state] * 100
    source = ColumnDataSource(new_vacc)
    dates = source.data['date'].tolist()
    p = figure(x_range=dates, title= f'Percentage of People Fully Vaccinated in {state} for last 7 days recorded')
    p.line(x='date', y='percentage', source=source, legend_label="Number of Vaccinations", line_width=8,line_color="green")
    p.add_tools(HoverTool())
    p.yaxis.axis_label = 'Percentage of People Fully Vaccinated (%)'
    p.xaxis.axis_label = 'Date (Day.Month.Year)'
    p.legend.location = "top_left"

    show(p)
    return p

line_chart_2(state)

Figure(id='1112', ...)

In [24]:
state = 'Vermont'

In [25]:
'''We have to make a note that 2 shots need to be given for a person to be considered fully vaccinated. 

Although Vermon has less shots given to people in that state. The Amount of people is way smaller than
Alabama so only 63% of the population has already been given'''

line_chart_1(state)

Figure(id='1229', ...)

In [26]:
line_chart_2(state)

Figure(id='1354', ...)